<a href="https://colab.research.google.com/github/akashnarwal7/First_repo/blob/master/InsuranceClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/balanced_insurance_dataset.csv")

# Display first few rows
df.head()


,Field Name,label
0,policyholder name,3
1,contact details phone number email,3
2,address,3
3,date of birth business establishment year,3
4,id proof aadhar pan passport etc,3


In [5]:
!pip install transformers datasets torch scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text
def tokenize_function(examples):
    return tokenizer(examples["Field Name"], padding="max_length", truncation=True)

# Convert dataset into Hugging Face format
from datasets import Dataset

dataset = Dataset.from_pandas(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split into train & test sets (90% train, 10% test)
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]


Map:   0%|          | 0/1944 [00:00<?, ? examples/s]

In [7]:
from transformers import BertForSequenceClassification

# Get unique labels
num_labels = len(df["label"].unique())

# Load BERT model for classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"  # Disables report logging (e.g., to WandB, TensorBoard)
)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
from transformers import Trainer
from transformers import DataCollatorWithPadding

# Define a data collator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-12-af25d31207c5>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.505243
2,No log,0.042974
3,0.834700,0.017340
4,0.834700,0.012190
5,0.022900,0.010965


TrainOutput(global_step=1095, training_loss=0.3929932404870856, metrics={'train_runtime': 869.2022, 'train_samples_per_second': 10.061, 'train_steps_per_second': 1.26, 'total_flos': 2301236721653760.0, 'train_loss': 0.3929932404870856, 'epoch': 5.0})

In [14]:
trainer.evaluate()


{'eval_loss': 0.010964542627334595,
 'eval_runtime': 5.5545,
 'eval_samples_per_second': 35.107,
 'eval_steps_per_second': 4.501,
 'epoch': 5.0}

In [15]:
import torch

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return predicted_class

# Test the model with an example input
predict("policy coverage details")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [18]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the correct device
model.to(device)

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)  # Move input to device
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return predicted_class

# Test the model with an example input
predict("softwre enginner")


13

In [19]:
# Evaluate the model on the test dataset
trainer.evaluate(test_dataset)


{'eval_loss': 0.010964542627334595,
 'eval_runtime': 5.9926,
 'eval_samples_per_second': 32.54,
 'eval_steps_per_second': 4.172,
 'epoch': 5.0}

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Run evaluation with the custom metric function
results = trainer.evaluate(test_dataset, metric_key_prefix="test")

print("Test Accuracy:", results["test_accuracy"])
print("Test Precision:", results["test_precision"])
print("Test Recall:", results["test_recall"])
print("Test F1 Score:", results["test_f1"])


KeyError: 'test_accuracy'

In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Associate the compute_metrics function with the Trainer
trainer.compute_metrics = compute_metrics

# Run evaluation with the custom metric function and metric key prefix
results = trainer.evaluate(test_dataset, metric_key_prefix="test")

print("Test Accuracy:", results["test_accuracy"])
print("Test Precision:", results["test_precision"])
print("Test Recall:", results["test_recall"])
print("Test F1 Score:", results["test_f1"])

Test Accuracy: 1.0
Test Precision: 1.0
Test Recall: 1.0
Test F1 Score: 1.0


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return predicted_class

# Example Test Cases
print("Prediction 1:", predict("policy coverage details"))
print("Prediction 2:", predict("customer claim "))
print("Prediction 3:", predict("software enginner"))



In [33]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return predicted_class

# Example Test Cases
print("Prediction 1:", predict("policy coverage details"))
print("Prediction 2:", predict("Death"))
print("Prediction 3:", predict("my name"))


Prediction 1: 12
Prediction 2: 17
Prediction 3: 3


In [34]:
new_test_data = [
    {"Field Name": "policy premium calculation", "Actual Label": "Premium"},
    {"Field Name": "customer claim rejection reason", "Actual Label": "Claim"},
    {"Field Name": "agent commission details", "Actual Label": "Agent"},
    {"Field Name": "insured vehicle registration number", "Actual Label": "Vehicle"},
    {"Field Name": "policyholder address update", "Actual Label": "Policyholder"},
    {"Field Name": "fraudulent claim detection", "Actual Label": "Fraud"},
    {"Field Name": "insurance coverage duration", "Actual Label": "Coverage"},
    {"Field Name": "emergency contact details", "Actual Label": "Contact"},
    {"Field Name": "renewal reminder notifications", "Actual Label": "Renewal"},
    {"Field Name": "hospital network coverage", "Actual Label": "Network"},
]


import pandas as pd
import torch
from sklearn.metrics import accuracy_score

# Move model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to predict label
def predict_label(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return label_mapping[predicted_class]  # Convert class index to label name

# Get predictions
predicted_labels = [predict_label(item["Field Name"]) for item in new_test_data]
actual_labels = [item["Actual Label"] for item in new_test_data]

# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

# Create DataFrame for detailed evaluation
evaluation_df = pd.DataFrame({
    "Field Name": [item["Field Name"] for item in new_test_data],
    "Actual Label": actual_labels,
    "Predicted Label": predicted_labels
})

# Print results
print(evaluation_df)
print("\nModel Accuracy on New Data: {:.2f}%".format(accuracy * 100))


NameError: name 'label_mapping' is not defined